### Torch observations
- for configure_optimizers

In [1]:
import torch
import torch.nn as nn

kernel_size = 3*3
fan_in = 3 * kernel_size
out_channels = 64
out_channels2 = 128

class Machine (nn.Module):    
    def __init__(self):
        super().__init__()
        self.first = nn.Linear (fan_in, out_channels, bias=True)
        self.second = nn.Conv2d (out_channels, out_channels2, 3, 1, 1)


model = Machine ()
num_paramters = sum (p.numel() for p in model.parameters())
num_paramters


75648

In [2]:
# 1792
# outchannels, in_channels, kernelsize0, kernelsize1

In [3]:
paramdict = {pn:p for pn,p in model.named_parameters()}

In [4]:
len(paramdict) # weight bias weight bias

4

In [5]:
paramdict['first.weight'].shape

torch.Size([64, 27])

In [6]:
paramdict['second.weight'].shape

torch.Size([128, 64, 3, 3])

In [7]:
paramdict['second.bias'].shape

torch.Size([128])

In [8]:
a = torch.zeros (768)
b = torch.zeros(768)
print (f"a_var:{a.var()}, a_std:{a.std()}, a_mean:{a.mean()}")
for i in range(100):
    a += (100)**-0.5 * torch.randn(768)
    b += torch.randn(768)

a.std(), b.std(), b.var()

a_var:0.0, a_std:0.0, a_mean:0.0


(tensor(0.9970), tensor(10.1498), tensor(103.0192))

In [9]:
in_channels = 3
out_channels = 64
kernal_size = 3
module = nn.Conv2d (in_channels, out_channels, kernal_size)


In [10]:
module.weight.shape
torch.nn.init._calculate_correct_fan(module.weight, mode='fan_in')

27

In [11]:
a = torch.randn(8)

In [12]:
a.shape

torch.Size([8])

In [13]:
b=a[8:16]

In [14]:
x = b.size()

In [15]:
x

torch.Size([0])

In [16]:
a = torch.randn(8)

In [17]:
b=a[8:10]

In [18]:
loaded_shard = torch.ones (64, 3, 256, 256)


In [19]:
len(loaded_shard)

64

In [20]:
import torch
from decoder import Decoder, DecoderConfig

dec = Decoder (DecoderConfig)

for pn,p in dec.named_parameters():
    print (f"{pn}:{p.shape}")

model.0.weight:torch.Size([512, 512, 3, 3])
model.0.bias:torch.Size([512])
model.1.conv_projection.weight:torch.Size([512, 512, 3, 3])
model.1.conv_projection.bias:torch.Size([512])
model.1.block.0.group_norm.weight:torch.Size([512])
model.1.block.0.group_norm.bias:torch.Size([512])
model.1.block.2.weight:torch.Size([512, 512, 3, 3])
model.1.block.2.bias:torch.Size([512])
model.1.block.3.group_norm.weight:torch.Size([512])
model.1.block.3.group_norm.bias:torch.Size([512])
model.2.group_norm.group_norm.weight:torch.Size([512])
model.2.group_norm.group_norm.bias:torch.Size([512])
model.2.conv_attention.weight:torch.Size([1536, 512, 1, 1])
model.2.conv_attention.bias:torch.Size([1536])
model.2.conv_projection.weight:torch.Size([512, 512, 1, 1])
model.2.conv_projection.bias:torch.Size([512])
model.3.conv_projection.weight:torch.Size([512, 512, 3, 3])
model.3.conv_projection.bias:torch.Size([512])
model.3.block.0.group_norm.weight:torch.Size([512])
model.3.block.0.group_norm.bias:torch.Size

VQGan(
  (encoder): Encoder(
    (initial_conv): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (model): Sequential(
      (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ResidualBlock(
        (conv_projection): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (block): Sequential(
          (0): GroupNorm(
            (group_norm): GroupNorm(32, 128, eps=1e-06, affine=True)
          )
          (1): Swish()
          (2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): GroupNorm(
            (group_norm): GroupNorm(32, 256, eps=1e-06, affine=True)
          )
          (4): Swish()
          (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (channel_up): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
      )
      (2): DownSampleBlock(
        (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
      )
      (3): 

In [1]:
from vqgan import VQGan
from encoder import EncoderConfig
from decoder import DecoderConfig
from quantizer import QuantizerConfig
import torch



gan = VQGan (EncoderConfig, QuantizerConfig, DecoderConfig)
gan.eval()
gan.to('cuda')
with torch.no_grad():
    x = torch.randn (4, 3, 256, 256).to('cuda')
    y = gan.encoder(x)
    loss, zq, encoding_indices = gan.quantizer(y)


In [2]:
with torch.no_grad():
    out = zq
    for i in range (len(gan.decoder.model)-1):
        out = gan.decoder.model[i](out)
        temp = out.mean()
        #temp.backward()
        print ("done",i)

done 0
done 1
done 2
done 3
done 4
done 5
done 6
done 7
done 8
done 9
done 10
done 11
done 12
done 13
done 14


In [3]:
out.requires_grad

False

In [4]:
#out.requires_grad_()


In [5]:
out.requires_grad

False

In [6]:
final = gan.decoder.model[-1](out)

In [7]:
final.shape


torch.Size([4, 3, 256, 256])

In [8]:
y= final.mean()

In [9]:
y.backward()

In [17]:
import torch.nn.functional as F
from lpips import LPIPS
vgg = LPIPS().eval()
x = torch.randn (4,3,256,256)
decoded_image = x
reconstruction_loss = F.mse_loss(x, decoded_image, reduction='none')
reconstruction_loss = reconstruction_loss.mean(dim=(1,2,3))

perc_loss = vgg (x, decoded_image)

c:\Users\Zen\miniconda3\envs\tr\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Zen\miniconda3\envs\tr\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
e:\Research\vincentVanGogh\lpips.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-mod

In [18]:
x.shape, reconstruction_loss.shape, perc_loss.shape

(torch.Size([4, 3, 256, 256]), torch.Size([4]), torch.Size([4, 1, 1, 1]))

In [3]:
from quantizer import Quantizer, QuantizerConfig
import torch
import torch.nn.functional as F

quantizer = Quantizer (QuantizerConfig)

ze =torch.randn (4, 512, 16, 16)
qloss, quantized, indices= quantizer(ze)

In [4]:
quantized.shape

torch.Size([4, 512, 16, 16])

In [56]:
vocab = torch.randn (256, 512)

In [57]:
(ze**2).shape, (vocab**2).shape

(torch.Size([4, 512, 16, 16]), torch.Size([256, 512]))

In [58]:
ze = ze.permute(0, 2, 3, 1).contiguous()
ze = ze.view (-1, 512)
ze.shape

torch.Size([1024, 512])

In [59]:
# a -b 2 = a2 + b2 - 2 a b
ze_squared = torch.sum (ze**2, dim=1, keepdim=True)
vocab_squared = torch.sum (vocab**2, dim=1, keepdim=True)
ze_squared.shape, vocab_squared.shape

(torch.Size([1024, 1]), torch.Size([256, 1]))

In [60]:
#BHW C, C,VS
ab2 = ze@vocab.transpose(0,1)
ab2.shape

torch.Size([1024, 256])

In [4]:
import torch

In [5]:
latent_dim = 512
vocab_size = 256
embedding = torch.randn(vocab_size, latent_dim)
z =torch.randn (4, latent_dim, 16, 16)
z.shape

torch.Size([4, 512, 16, 16])

In [3]:
z = z.permute(0, 2, 3, 1).contiguous()
z_flattened = z.view(-1, latent_dim)

d = torch.sum(z_flattened**2, dim=1, keepdim=True) + \
torch.sum(embedding**2, dim=1) - \
2*(torch.matmul(z_flattened, embedding.t()))


In [6]:
(torch.sum(z_flattened**2, dim=1, keepdim=True)).shape

torch.Size([1024, 1])

In [7]:
(torch.sum(embedding**2, dim=1)).shape

torch.Size([256])

In [2]:
import torch
import torch.nn.functional as F

In [3]:
x = torch.randn (4,3,256,256)
x2 = torch.randn (4,3,256,256)
reconstruction_loss = torch.abs(x - x2)
reconstruction_loss = reconstruction_loss.to('cuda')

In [4]:
reconstruction_loss.shape

torch.Size([4, 3, 256, 256])

In [5]:
from lpips import LPIPS
m = LPIPS().eval()
x= x.to('cuda')
x2= x2.to('cuda')
m.to('cuda')

prec = m(x,x2)

c:\Users\Zen\miniconda3\envs\tr\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Zen\miniconda3\envs\tr\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
e:\Research\vincentVanGogh\lpips.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-mod

In [6]:
perc_rec = prec + reconstruction_loss
perc_rec.shape

torch.Size([4, 3, 256, 256])